In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/minidev/mini_dev_postgresql.json
/kaggle/input/minidev/dev_tables.json


In [2]:
!pip install transformers==4.51.3 accelerate sqlparse sql-metadata auto_gptq optimum --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 77.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 77.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 100.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━

In [3]:
import json

def load_dataset(filename):
    with open(filename, 'r') as f:
        j = json.load(f)
    return j

dataset = load_dataset("/kaggle/input/minidev/mini_dev_postgresql.json")

In [4]:
import json

def convert_json_to_schema(filename):
    """Convert a tables.json file into a python dictionary keeping the relevant schema information."""
    databases = {}
    with open(filename, "r") as tables_json_file:
        tables_json = json.load(tables_json_file)
        for table_data in tables_json:
            database_schema = []

            table_names = table_data["table_names_original"]
            for table_name in table_names:
                table_schema = {
                    "name": table_name,
                    "columns": [],
                    "primary_keys": [],
                    "foreign_keys": []
                }
                database_schema.append(table_schema)

            column_names = table_data["column_names_original"]
            column_datatypes = table_data["column_types"]
            for [table_index, column_name], datatype in zip(column_names[1:], column_datatypes[1:]):
                column_name = column_name.replace(" ", "_")
                column_name = column_name.replace("-", "_")

                if datatype == "datetime":
                    datatype = "timestamp"
                    
                column = {
                    "name": column_name,
                    "type": datatype
                }
                database_schema[table_index]["columns"].append(column)

            for primary_key in table_data["primary_keys"]:
                if type(primary_key) is int:
                    primary_key = [primary_key,]

                for key in primary_key:
                    table_index = column_names[key][0]
                    column_name = column_names[key][1]
                    database_schema[table_index]["primary_keys"].append(column_name)

            for foreign_key in table_data["foreign_keys"]:
                column_name = column_names[foreign_key[0]][1]
                reference_table = database_schema[column_names[foreign_key[1]][0]]["name"]
                reference_column = column_names[foreign_key[1]][1]
                database_schema[column_names[foreign_key[0]][0]]["foreign_keys"].append({
                    "column": column_name,
                    "references": f"{reference_table}({reference_column})"
                })

            databases[table_data["db_id"]] = database_schema

    return databases


db = convert_json_to_schema("/kaggle/input/minidev/dev_tables.json")

print("\n".join(list(db.keys())[:10]))

debit_card_specializing
financial
formula_1
california_schools
card_games
european_football_2
thrombosis_prediction
toxicology
student_club
superhero


In [5]:
def get_normalized_create_statement(database_schema, filtered_tables=None):
    """Return database_schema as PostgreSQL normalized create table statements"""
    schema_text = []
    for table in database_schema:
        if filtered_tables is None or table['name'] in filtered_tables:
            create_table_text = f"create table {table['name']} ("
            
            # Columns
            columns = []
            for column in table["columns"]:
                column_def = f"{column['name']} {column['type'].lower()}"
                columns.append(column_def)
            create_table_text += ("\n" + ",\n".join(columns))
            
            # Primary keys
            if table["primary_keys"]:
                create_table_text += (f",\nprimary key ({', '.join(table['primary_keys'])})")
            
            # Foreign keys
            for fk in table["foreign_keys"]:
                create_table_text += (f",\nforeign key ({fk['column']}) references {fk['references']}")

            create_table_text += "\n);"

            schema_text.append(create_table_text)  
    
    return "\n".join(schema_text)

print(get_normalized_create_statement(db["formula_1"]))

create table circuits (
circuitId integer,
circuitRef text,
name text,
location text,
country text,
lat real,
lng real,
alt integer,
url text,
primary key (circuitId)
);
create table constructors (
constructorId integer,
constructorRef text,
name text,
nationality text,
url text,
primary key (constructorId)
);
create table drivers (
driverId integer,
driverRef text,
number integer,
code text,
forename text,
surname text,
dob date,
nationality text,
url text,
primary key (driverId)
);
create table seasons (
year integer,
url text,
primary key (year)
);
create table races (
raceId integer,
year integer,
round integer,
circuitId integer,
name text,
date date,
time text,
url text,
primary key (raceId),
foreign key (circuitId) references circuits(circuitId),
foreign key (year) references seasons(year)
);
create table constructorResults (
constructorResultsId integer,
raceId integer,
constructorId integer,
points real,
status text,
primary key (constructorResultsId),
foreign key (constructor

In [6]:
from sql_metadata import Parser

def create_prompt(d):
    schema = get_normalized_create_statement(db[d["db_id"]], Parser(d["SQL"]).tables)
    question = d["question"] + (" " + d["evidence"] if d["evidence"] else "")
    return f"""You are a data science expert. Below, you are provided with a database schema and a natural language question. Your task is to understand the schema and generate a valid PostgreSQL query to answer the question.
    
Database Schema:
{schema}

Question:
{question}

Instructions:
- Make sure you only output the information that is asked in the question. If the question asks for a specific column, make sure to only include that column in the SELECT clause, nothing more.
- The generated query should return all of the information asked in the question without any missing or extra information.
- Before generating the final SQL query, please think through the steps of how to write the query. Do all the explanation before generating the final query.
- Make sure to check the datatypes of the columns. For Example: if Date column has text datatype, do not use date functions on it, use string functions.
- If you think some table information is missing or the database schema provided has no relevance with the question, do not answer with any SQL query.

Take a deep breath and think step by step to find the correct SQL query.
"""

print(create_prompt(dataset[0]))

You are a data science expert. Below, you are provided with a database schema and a natural language question. Your task is to understand the schema and generate a valid PostgreSQL query to answer the question.
    
Database Schema:
create table customers (
CustomerID integer,
Segment text,
Currency text,
primary key (CustomerID)
);

Question:
What is the ratio of customers who pay in EUR against customers who pay in CZK? ratio of customers who pay in EUR against customers who pay in CZK = count(Currency = 'EUR') / count(Currency = 'CZK').

Instructions:
- Make sure you only output the information that is asked in the question. If the question asks for a specific column, make sure to only include that column in the SELECT clause, nothing more.
- The generated query should return all of the information asked in the question without any missing or extra information.
- Before generating the final SQL query, please think through the steps of how to write the query. Do all the explanation b

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct-GPTQ-Int4"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.eval()

config.json: 0.00B [00:00, ?B/s]

2025-07-15 12:59:39.899529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752584380.252629      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752584380.359821      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): Qwen2MLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3584, out_features=152064, bias=False)
)

In [8]:
prompts_with_chat_template = []
for d in dataset:
    prompt = create_prompt(d)
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    prompts_with_chat_template.append(text)

print(prompts_with_chat_template[0])

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
You are a data science expert. Below, you are provided with a database schema and a natural language question. Your task is to understand the schema and generate a valid PostgreSQL query to answer the question.
    
Database Schema:
create table customers (
CustomerID integer,
Segment text,
Currency text,
primary key (CustomerID)
);

Question:
What is the ratio of customers who pay in EUR against customers who pay in CZK? ratio of customers who pay in EUR against customers who pay in CZK = count(Currency = 'EUR') / count(Currency = 'CZK').

Instructions:
- Make sure you only output the information that is asked in the question. If the question asks for a specific column, make sure to only include that column in the SELECT clause, nothing more.
- The generated query should return all of the information asked in the question without any missing or extra information.
- Before

In [9]:
def extract_query(response): # assuming the last code block with ```sql ``` has the final query
    return response[response.rfind("```sql\n") + 7:response.rfind("```")]

In [ ]:
from tqdm.auto import tqdm

try:
    with open("output.json", 'r+') as f:
        responses = json.load(f)
except FileNotFoundError:
    responses = []
for text in tqdm(prompts_with_chat_template[len(responses):], desc="Generating"):
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=1024
        )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    responses.append({"response": response, "SQL": extract_query(response)})
    with open("output.json", 'w+') as f:
        json.dump(responses, f, indent=4)

print(len(responses))
print(responses[0]['SQL'])

Generating:   0%|          | 0/100 [00:00<?, ?it/s]